In [4]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

# Import the CRUD module
from CRUD import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# Authenticate to the AAC database
username = "aacuser"
password = "userPassword1"
shelter = AnimalShelter(username, password)


# Retrieve all data from the database 
df = pd.DataFrame.from_records(shelter.read_all({}))


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

# Add in Grazioso Salvare’s logo
image_filename = 'Grazioso_Salvare_Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Position logo, dashboard title, and unique identifier
app.layout = html.Div([
    # Add image with anchor to snhu.edu
    html.Center(html.A([html.Img(height='150', width='150', 
                        src='data:image/png;base64,{}'.format(encoded_image.decode()))],
                href='https://www.snhu.edu')),
    html.Center(html.B(html.H1('Grazioso Salvare Rescue Animals'))),
    html.H6("Dashboard Author: Sarah Spence"),
    html.Hr(),
    html.Div(
        # Add radio item for rescue dog filter options
        dcc.RadioItems(
            id="filter-type",
            options=[
                {'label': 'Water Rescue', 'value': 'Water'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain'},
                {'label': 'Disaster Rescue or Individual Tracking', 'value': 'Disaster'},
                {'label': 'Reset Filters', 'value': 'Reset'}
            ],
            value='Reset',
            labelStyle={'display': 'inline-block'}
        )
    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ], 
        style_table={'overflowX':'scroll'},
        data=df.to_dict('records'),
        # Set up the features for the interactive data table to make it user-friendly for the client
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable='multi',
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,   
    ),
    html.Br(),
    html.Hr(),
    # This sets up the dashboard so the pie chart and geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    """Filter rescue dogs based on the rescue type."""
    if filter_type == 'Water':
        df = pd.DataFrame.from_records(shelter.filtered_rescue_dogs("Water"))
        
    elif filter_type == 'Mountain':
        df = pd.DataFrame.from_records(shelter.filtered_rescue_dogs("Mountain"))
        
    elif filter_type == 'Disaster':
        df = pd.DataFrame.from_records(shelter.filtered_rescue_dogs("Disaster"))
        
    elif filter_type == 'Reset':
        df = pd.DataFrame.from_records(shelter.read_all({}))
                
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns[1:]]
    data=df.to_dict('records')
            
    return (data,columns)

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])
def update_styles(selected_columns):
    """Update style"""
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(data):
    """Create a pie chart based on the rescue dog type"""
    df = pd.DataFrame.from_dict(data)
    fig = px.pie(df,
                 values=df.age_upon_outcome_in_weeks,
                 names=df.breed,
                 title="Rescue Animals by Breed",
                 width=600,
                 height=500)
    return [
        dcc.Graph(            
            figure = fig
        )    
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    """Add the geolocation chart"""
    dff = pd.DataFrame.from_dict(viewData)
    # Austin Texas is at [[30.75, -97.48]
    return [
        dl.Map(style={'width': '625px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ] 


app